In [1]:
import sys
sys.version
sys.version_info

sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)

In [3]:
import nltk
from nltk import word_tokenize

In [63]:
import glob2
filenames = glob2.glob('./Neg/*.txt')  # list of all .txt files in the directory

with open('all_neg.txt', 'w') as f:
    for file in filenames:
        with open(file) as infile:
            f.write(infile.read()+'\n')
       

In [64]:
neg_txt = open('all_neg.txt')
neg = neg_txt.read()
length_neg = len(neg.split('\n'))
print(length_neg)

56


In [188]:
filenames = glob2.glob('./Pos/*.txt')  # list of all .txt files in the directory

with open('all_pos.txt', 'w') as f:
    for file in filenames:
        with open(file) as infile:
            f.write(infile.read()+'\n')

pos_txt = open('all_pos.txt')
pos = pos_txt.read()
length_pos = len(pos.split('\n'))
print((pos[:400]))

I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. Thi


In [66]:
# count number of tokens in negative reviews (including stop words)

In [237]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

neg = neg.lower()
#tokens_neg = word_tokenize(neg)
# including regular expressions
tokens_neg = tokenizer.tokenize(neg)
print("# of negative tokens", len(tokens_neg))

# of negative tokens 15369


In [79]:
# count number of tokens in positive reviews (including stop words)

In [84]:


pos = pos.lower()
#tokens_pos = word_tokenize(pos)
# including regular expressions
tokens_pos = tokenizer.tokenize(pos)
len(tokens_pos)

11824

In [180]:
#remove English stopwords from both texts
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))
tokens_neg_clean = [word for word in tokens_neg if word not in stops]
tokens_pos_clean = [word for word in tokens_pos if word not in stops]
print(len(tokens_neg_clean))
print(len(tokens_pos_clean))

8101
6092


# Answer to question #1 = 14193

In [124]:
total_tokens = tokens_neg_clean + tokens_pos_clean
print(len(total_tokens))
print(total_tokens[:10])

14193
['mr', 'costner', 'dragged', 'movie', 'far', 'longer', 'necessary', 'aside', 'terrific', 'sea']


# Answer to question #2 = 4438

In [87]:
len(set(total_tokens))

4438

# Question 3

In [127]:
freq_distr_unigram = nltk.FreqDist(total_tokens)
print(freq_distr_unigram.most_common(10))
print(freq_distr_unigram['movie'])

[('br', 442), ('film', 206), ('movie', 195), ('one', 120), ('like', 95), ('good', 67), ('great', 66), ('really', 63), ('see', 60), ('story', 60)]
195


In [115]:
bigrams = nltk.bigrams(tokens_pos_clean+tokens_neg_clean)
#compute frequency distribution for all the bigrams in the text
freq_distr_bigram = nltk.FreqDist(bigrams)
top10_bigrams_freq = freq_distr_bigram.most_common(10)
#just the bigrams without the freq-cy
top10_bigrams = [x[0] for x in top10_bigrams_freq]
print('Top 10 bigrams and their frequencies:')
print(top10_bigrams_freq)


Top 10 bigrams and their frequencies:
[(('br', 'br'), 222), (('coast', 'guard'), 18), (('kevin', 'costner'), 14), (('ashton', 'kutcher'), 12), (('lackawanna', 'blues'), 12), (('tom', 'courtenay'), 10), (('albert', 'finney'), 10), (('epatha', 'merkerson'), 9), (('cover', 'girl'), 9), (('jacqueline', 'hyde'), 9)]


In [130]:
w1 = 'coast'
w2 = 'guard'
print(freq_distr_bigram[(w1, w2)])

18


In [116]:
trigrams = nltk.trigrams(tokens_pos_clean+tokens_neg_clean)
#compute frequency distribution for all the bigrams in the text
freq_distr_trigrams = nltk.FreqDist(trigrams)
top10_trigrams_freq = freq_distr_trigrams.most_common(10)
print('Top 10 trigrams and their frequencies:')
#just the trigrams without the frequencies
top10_trigrams = [x[0] for x in top10_trigrams_freq]
#print(top10_trigrams)
print(top10_trigrams_freq)

Top 10 trigrams and their frequencies:
[(('br', 'br', 'movie'), 6), (('br', 'br', 'overall'), 6), (('br', 'br', 'great'), 5), (('movie', 'br', 'br'), 5), (('br', 'br', 'film'), 5), (('ruben', 'santiago', 'hudson'), 5), (('br', 'br', 'also'), 4), (('u', 'coast', 'guard'), 4), (('end', 'br', 'br'), 4), (('albert', 'finney', 'tom'), 4)]


# Question 4

In [134]:
print(freq_distr_unigram['movie']/freq_distr_bigram[('br', 'br')])
print(freq_distr_unigram['film']/freq_distr_bigram[('br', 'br')])

0.8783783783783784
0.9279279279279279


In [139]:
probs = {}
def prob_of_3rdword(w1, w2, w3):
    probs[(w3, w1, w2)] = freq_distr_unigram[w3]/freq_distr_bigram[(w1, w2)]
    print(probs[(w3, w1, w2)])

In [140]:
prob_of_3rdword('br', 'br', 'movie')

0.8783783783783784


In [181]:
# Laplace smoothing (parameter = 0.1)

In [160]:
prev_words = {}
for t in total_tokens[1: len(total_tokens) -1]:
    if t not in prev_words:
        prev_words[t] = 1
    else:
        prev_words[t] += 1
        
bigrams = [(total_tokens[n], total_tokens[n+1]) for n in range(1, len(total_tokens)-1)]

In [161]:
print(bigrams[:10])

[('costner', 'dragged'), ('dragged', 'movie'), ('movie', 'far'), ('far', 'longer'), ('longer', 'necessary'), ('necessary', 'aside'), ('aside', 'terrific'), ('terrific', 'sea'), ('sea', 'rescue'), ('rescue', 'sequences')]


In [165]:
bigram_counts_laplace = {}
for bigram in bigrams:
    if bigram not in bigram_counts_laplace:
        bigram_counts_laplace[bigram] = 1.1
    else:
        bigram_counts_laplace[bigram]+=1
bigram_counts_laplace[('coast', 'guard')]

18.1

In [170]:
bigram_probs_laplace = {}
for k,v in bigram_counts_laplace.items():
    bigram_probs_laplace[(k[1], k[0])] = v / (prev_words[k[0]] + 0.1*len(prev_words))

In [171]:
bigram_probs_laplace[('guard', 'coast')]

0.03903385809790813

In [196]:
bigrams[1]

('dragged', 'movie')

In [200]:
prev_2words = {}
for t in bigrams:
    if t not in prev_2words:
        prev_2words[t] = 1
    else:
        prev_2words[t] += 1

In [201]:
prev_2words[('costner', 'movie')]        

3

In [198]:
trigram_counts_laplace = {}
for trigram in trigrams:
    if trigram not in trigram_counts_laplace:
        trigram_counts_laplace[trigram] = 1.1
    else:
        trigram_counts_laplace[trigram]+=1
trigram_counts_laplace[('br','br', 'movie')]

6.1

In [210]:
trigram_counts_laplace

{('costner', 'dragged', 'movie'): 1.1,
 ('dragged', 'movie', 'far'): 1.1,
 ('movie', 'far', 'longer'): 1.1,
 ('far', 'longer', 'necessary'): 1.1,
 ('longer', 'necessary', 'aside'): 1.1,
 ('necessary', 'aside', 'terrific'): 1.1,
 ('aside', 'terrific', 'sea'): 1.1,
 ('terrific', 'sea', 'rescue'): 1.1,
 ('sea', 'rescue', 'sequences'): 1.1,
 ('rescue', 'sequences', 'care'): 1.1,
 ('sequences', 'care', 'characters'): 1.1,
 ('care', 'characters', 'us'): 1.1,
 ('characters', 'us', 'ghosts'): 1.1,
 ('us', 'ghosts', 'closet'): 1.1,
 ('ghosts', 'closet', 'costner'): 1.1,
 ('closet', 'costner', 'character'): 1.1,
 ('costner', 'character', 'realized'): 1.1,
 ('character', 'realized', 'early'): 1.1,
 ('realized', 'early', 'forgotten'): 1.1,
 ('early', 'forgotten', 'much'): 1.1,
 ('forgotten', 'much', 'later'): 1.1,
 ('much', 'later', 'time'): 1.1,
 ('later', 'time', 'care'): 1.1,
 ('time', 'care', 'character'): 1.1,
 ('care', 'character', 'really'): 1.1,
 ('character', 'really', 'care'): 1.1,
 ('re

In [218]:
trigram_probs_laplace = {}
for k,v in trigram_counts_laplace.items():
    trigram_probs_laplace[(k[2],k[0], k[1])] = v / (prev_2words[(k[0], k[1])] + 0.1*len(prev_2words))
    #print(prev_2words[(k[0], k[1])])

In [219]:
def get_trigram_prob(w1, w2, w3):
    return trigram_probs_laplace[w3, w1,w2]

In [236]:
#trigram_probs_laplace

In [238]:
#print(freq_distr_trigrams.most_common(20))
print(get_trigram_prob('costner', 'dragged', 'movie'))
print(get_trigram_prob('overconfident', 'ashton', 'kutcher'))
print(get_trigram_prob('characters', 'us', 'ghosts'))
print(get_trigram_prob('far','longer', 'necessary'))
print((get_trigram_prob('care', 'characters', 'us')))

0.0008502744067403571
0.0008502744067403571
0.0008502744067403571
0.0008502744067403571
0.0008496176720475786
